In [1]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class SqueezeExcitationBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):#通道数，reduction=16为降维系数16
        super(SqueezeExcitationBlock, self).__init__()#初始化
        self.avg_pool = nn.AdaptiveAvgPool2d(1)#nn.AdaptiveAvgPool2d(1) 将对每个通道进行平均池化，并将输出大小设置为 1x1×C 如果括号里面是2则变成2×2×C
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction),#全连接层 变成1×1×C/r
            nn.ReLU(inplace=True),#激活函数
            nn.Linear(in_channels // reduction, in_channels),#全连接层 变成1×1×C
            nn.Sigmoid()#0-1权重
        )
    def forward(self, x):
        b, c, _, _ = x.size()#从输入的 x 中取出 batch size 和通道数，而忽略高度和宽度的信息。
        y = self.avg_pool(x).view(b, c)#view(b,c)是将原来的四维张量改变为二维张量，第一维度是批量大小b，第二是通道数c
        y = self.fc(y).view(b, c, 1, 1)#首先通过一个全连接层用于学习权重，在讲处理过的y更改为4维张量，形状为(batch_size, channels, 1, 1)
        return x * y.expand_as(x)#先将y扩展为和x相同的形状，然后执行逐元素乘法操作。实际上是将学习到的每个通道的权重应用到输入x的对应通道上

class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_channels),
            SqueezeExcitationBlock(in_channels)
        )
    def forward(self, x):
        residual = x
        out = self.block(x)
        out += residual
        out = nn.ReLU(inplace=True)(out)
        return out


class SubpixelConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, scale_factor):
        super(SubpixelConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels * scale_factor ** 2, kernel_size=3, stride=1, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(scale_factor)

    def forward(self, x):
        x = self.conv(x)
        x = self.pixel_shuffle(x)
        return x


class SuperResolutionModel(nn.Module):
    def __init__(self):
        super(SuperResolutionModel, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4),
            nn.ReLU(inplace=True)
        )
        self.block2 = nn.Sequential(
            *[ResidualBlock(64) for _ in range(16)]
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
        )
        self.block4 = SubpixelConv2d(64, 3, 2)  
        self.block5 = nn.Sequential(
            nn.Conv2d(3, 3, kernel_size=9, stride=1, padding=4),  
            nn.Sigmoid()
        )#slef.block5负责对上一步的输出调整到目标的尺寸和通道数，这里是3通道的RGB图像。

    def forward(self, x):
        block1 = self.block1(x)
        block2 = self.block2(block1)
        block3 = self.block3(block2)
        block3 += block1
        block4 = self.block4(block3)
        block5 = self.block5(block4)
        return block5


class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):#初始化数据集的，low_res_folder和high_res_folder分别代表了低分辨率图像和高分辨率图像的文件夹路径，transform是一个可选参数，用来定义是否对图像进行某种预处理操作
        self.low_res_folder = low_res_folder
        self.high_res_folder = high_res_folder
        self.transform = transform

        self.low_res_images = os.listdir(low_res_folder)#低分辨率图像样本路径
        self.high_res_images = os.listdir(high_res_folder)#高分辨率图像样本路径

    def __len__(self):
        return len(self.low_res_images)#返回数据集中的样本数，这里返回的低分辨率样本数

    def __getitem__(self, idx):#定义了如何根据索引（idx）获取一个样本
        low_res_image = Image.open(os.path.join(self.low_res_folder, self.low_res_images[idx]))#使用os获取低分辨率图像的路径，并用PIL库的Image.open函数打开它。
        high_res_image = Image.open(os.path.join(self.high_res_folder, self.high_res_images[idx]))

        if self.transform:#如果定义了transform，则返回时先对图像进行预处理操作，transform=None则不对图像进行预处理
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

def generate_and_save_images(model, device, class_name, low_res_folder, save_folder):
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])#定义了一个图像处理管道，该管道将PIL图像转化为PyTorch张量
    model.eval()#为模型设置为评估模式
    with torch.no_grad():#该语句用于临时关闭梯度计算，从而减少内存使用并加速计算
        for filename in os.listdir(low_res_folder):#遍历低分辨率图像文件夹中的每个文件
            low_res_image = Image.open(os.path.join(low_res_folder, filename))
            low_res_image = transform(low_res_image).unsqueeze(0).to(device)#将PIL图像转化为张量，增加一个批处理维度，然后将数据移动到指定的设备上，unsqueeze(0)的作用是在张量的第一个维度（下标为0的位置）增加一个新的维度。
            high_res_image = model(low_res_image)#生成高分辨率
            high_res_image = high_res_image.squeeze(0).permute(1,2,0).cpu().numpy()#移除批处理维度，改变张量的维度顺序，然后将数据移动到CPU并转换为numpy数组。

            # 转换数据范围为0-255并保存
            high_res_image = (high_res_image * 255).astype(np.uint8)#数据类型转换为无符号8位整数，这是图像通常使用的数据类型
            high_res_image = Image.fromarray(high_res_image)#将numpy数组转换为PIL图像。

            save_path = os. path.join(save_folder, filename)
            high_res_image.save(save_path)

    

def train_and_generate(class_name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#运行配置

    transform = transforms.Compose([
        transforms.ToTensor(),
    ])#定义了一个图像处理管道，该管道将PIL图像转化为PyTorch张量

    batch_size = 2

    low_res_folder = f'data/low200/{class_name}'
    high_res_folder = f'data/origin400/{class_name}'
    save_folder = f'new400/{class_name}'
    os.makedirs(save_folder, exist_ok=True)

    assert os.path.isdir(low_res_folder), f"{low_res_folder} does not exist."
    assert os.path.isdir(high_res_folder), f"{high_res_folder} does not exist."

    dataset = CustomDataset(low_res_folder, high_res_folder, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)#Shuffle表示打乱

    model = SuperResolutionModel().to(device)#使用模型
    criterion = nn.MSELoss()#MSE损失函数
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)#adam优化

    num_epochs = 20
    accumulation_steps = 4  # 选择适当的累积步数，使得 batch_size * accumulation_steps 等于原始的批量大小

    for epoch in range(num_epochs):
        for i, (low_res_images, high_res_images) in enumerate(dataloader):
            low_res_images = low_res_images.to(device)
            high_res_images = high_res_images.to(device)

            outputs = model(low_res_images)#输入为低分辨率
            loss = criterion(outputs, high_res_images)#输出高分辨率与实际高分辨率求损失

            loss = loss / accumulation_steps  
            loss.backward()  

            if (i+1) % accumulation_steps == 0:  # 在累积足够的梯度后更新权重
                optimizer.step()
                optimizer.zero_grad()

            if (i+1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item()}")

        torch.save(model.state_dict(), f'{class_name}_super_res.pth')
        
    # 在每个类别的训练结束后生成和保存高分辨率图像
    generate_and_save_images(model, device, class_name, low_res_folder, save_folder)



if __name__ == "__main__":
    # 使用两个GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs!")
        model = SuperResolutionModel()
        model = nn.DataParallel(model)  # 包装模型以支持多GPU训练
    else:
        model = SuperResolutionModel()

    model = model.to(device)
    for class_name in ['class1', 'class2', 'class3']:
        train_and_generate(class_name)


AssertionError: data/low200/class1 does not exist.

ModuleNotFoundError: No module named 'torch'